In [25]:
import requests  
from bs4 import BeautifulSoup
from datetime import datetime
from time import sleep
import pandas as pd
import fake_useragent
from fake_useragent import UserAgent
import random
from tqdm import tqdm

## Парсинг сайта ##

In [16]:
cars_data = pd.DataFrame()

In [27]:
for page in range(1, 21): # Задаем число просматриваемых страниц с объявлениями
    url = f'https://auto.ru/moskva/cars/used/?page={page}'
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    content = BeautifulSoup(response.content, 'html.parser')
    cars = content.find_all('div', {'class': 'ListingItem-module__description'})
    sleep(4.5 + random.random()) # Бездействуем, чтобы нас не забанили
    for car in tqdm(cars):
        try:
            car_info = dict() # Формирем словарь и информацией об автомобиле
            car_link = car.find('a', { 'class': ['Link', 'ListingItemTitle-module__link']}).get('href')
            response = requests.get(car_link, headers={'User-Agent': UserAgent().chrome})
            content = BeautifulSoup(response.content, 'html.parser')
            car_info['Link'] = car_link
            table = content.find_all('li', { 'class': 'CardInfoRow' })
            car_info['Title'] = content.find('h1').text.replace('\xa0', ' ')
            car_info['Price'] = content.find('span', 'OfferPriceCaption__price').text.replace('\xa0', ' ')
            car_info['Mark'] = content.find('div', {'class': ['InfoPopup','InfoPopup_theme_plain','InfoPopup_withChildren','BreadcrumbsPopup-module__BreadcrumbsPopup']}).find('a', { 'class': ['Link','Link_color_gray','CardBreadcrumbs__itemText']}).text.replace('\xa0', ' ')
            car_info['Placement_date'] = content.find('div', { 'class' : ['lgkqdBtNZnmDcmU0rHpEj__info-item']}).text.replace('\xa0', ' ')
            car_info['Views'] = content.find('div', { 'class' : ['lgkqdBtNZnmDcmU0rHpEj__info-item','lgkqdBtNZnmDcmU0rHpEj__views'], 'title' : 'Количество просмотров'}).text.replace('\xa0', ' ')
            for raw in table: 
                car_info[raw.find_all('span')[0].text] = raw.find_all('span')[1].text.replace('\xa0', ' ')
            sleep(4.5 + random.random())
            cars_data = cars_data.append(car_info, ignore_index=True) # Добавляем информацию об авто в датафрейм
        except:
            sleep(4.5 + random.random())
            continue

100%|██████████| 38/38 [03:43<00:00,  5.88s/it]


In [32]:
cars_data.shape # Смотрим на число автомобилей и признаков

(0, 0)

In [33]:
cars_data.to_csv('cars_data.csv', index=False, sep='\t') # На всякий пожарный сохраняем в файл

## Обработка данных ##

In [34]:
cars_data1 = pd.read_csv('cars_data.csv',delimiter='\t') 
cars_data1.head(2)

EmptyDataError: No columns to parse from file

Смотрим в каких столбцах есть пропуски

In [ ]:
cars_data1.isna().sum()

Удаляем столбцы, в которых слишком много пропусков

In [ ]:
drop_list = list() 
for i in range(len(cars_data1.isna().sum().values)):
    if cars_data1.isna().sum().values[i] / cars_data1.shape[0] > 0.7:
        drop_list.append(cars_data1.isna().sum().index[i])
cars_data1.drop(columns=drop_list, inplace=True)
drop_list

Корректируем столбцы, где хранятся числовые данные

In [ ]:
def money_correcter(raw):
    try:
        raw = raw.replace(' ', '')
        raw = raw[:raw.find('₽')]
        return raw
    except:
        return raw

def probeg_correcter(raw):
    try:
        raw = raw.replace(' ', '')
        raw = raw[:raw.find('км')]
        return raw
    except:
        return raw

In [ ]:
cars_data1['Налог'] = cars_data1['Налог'].apply(money_correcter)
cars_data1['Price'] = cars_data1['Price'].apply(money_correcter)
cars_data1['Пробег'] = cars_data1['Пробег'].apply(probeg_correcter)

In [ ]:
cars_data1['Налог'] = pd.to_numeric(cars_data1['Налог'])
cars_data1['Price'] = pd.to_numeric(cars_data1['Price'])
cars_data1['год выпуска'] = pd.to_numeric(cars_data1['год выпуска'])
cars_data1['Пробег'] = pd.to_numeric(cars_data1['Пробег'])

Оставшиеся пропуски заполняем

Вин заменим пустыми строками

In [ ]:
cars_data1['VIN'] = cars_data1['VIN'].fillna('')

Налог на среднее значение

In [ ]:
cars_data1['Налог'] = cars_data1['Налог'].fillna(cars_data1['Налог'].mean())

Обмен на самое частовстречаемое значение

In [ ]:
cars_data1["Обмен"].fillna(cars_data1['Обмен'].value_counts().index[0], inplace = True)

Пропусков стало меньше

In [ ]:
cars_data1.isna().sum()

## Новые признаки ##

In [ ]:
def age_definer(car_year):
    current_year = datetime.now().timetuple()[0]
    if current_year - car_year < 3:
        return 'Новая'
    if current_year - car_year < 6:
        return 'Средняя'
    else:
        return 'Старая'

def km_per_year(car_year, car_mileage):
    try:
        current_year = datetime.now().timetuple()[0]
        return car_mileage // (current_year - car_year + 1)
    except:
        return None

def total_views_definer(raw):
    try:
        return raw[:raw.find(' ')]
    except:
        return None

Присваиваем категории возраста для автомобиля

In [ ]:
cars_data1['Возраст'] = cars_data1['год выпуска'].apply(age_definer)
cars_data1['Возраст'][:6]

Присваем категорию сколько автомобиль ездил в среднем за год своего существования

In [ ]:
cars_data1['Пробег за год'] = cars_data1[['год выпуска','Пробег']].apply(lambda x: km_per_year(*x), axis=1)
cars_data1['Пробег за год'][:6]

Делаем категорию с общим числов просмотров автомобиля (до этого в столбце было строка вида *435 (116 сегодня)*)

In [ ]:
cars_data1['Total views'] = pd.to_numeric(cars_data1['Views'].apply(total_views_definer))
cars_data1['Total views'][:6]

## Графики ##

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

Сделаем график распределения машин по возрастным категориям в рамках фирмы. 
Построим для топ 3 фирм по числу объявлений, чтобы не нагромождать график.

In [ ]:
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

In [ ]:
ser = cars_data1['Mark'].value_counts()
mark_list = ser.iloc[:3].index.tolist()
grouped_data = cars_data1[cars_data1['Mark'].isin(mark_list)].groupby(['Mark', 'Возраст']).size()

all_marks = grouped_data.to_frame().reset_index()
first_mark = all_marks[all_marks['Mark'] == ser.iloc[:3].index.tolist()[0]]
second_mark = all_marks[all_marks['Mark'] == ser.iloc[:3].index.tolist()[1]]
third_mark = all_marks[all_marks['Mark'] == ser.iloc[:3].index.tolist()[2]]

first_name = first_mark['Mark'].values[0]
second_name = second_mark['Mark'].values[0]
third_name = third_mark['Mark'].values[0]

labels = first_mark['Возраст'].tolist()

first_mark = first_mark[0].tolist()
second_mark = second_mark[0].tolist()
third_mark = third_mark[0].tolist()


x = np.arange(len(labels))
width = 0.2 

fig, ax = plt.subplots()
rects1 = ax.bar(x - width, first_mark, width, label=first_name)
rects2 = ax.bar(x, second_mark, width, label=second_name)
rects3 = ax.bar(x + width, third_mark, width, label=third_name)

ax.set_ylabel('Количество машин')
ax.set_title('Количество машин в разных возрастных категориях')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

fig.tight_layout()

plt.show()

Посмотрим, какую долю объвлений занимает каждая фирма автомобилей. 
Подпишем в процентном соотношении, предварительно удалив те, у которых доля объявлений меньше 3% от общего числа, чтобы не нагромождать график

In [ ]:
cars_data1['Mark'].value_counts().loc[lambda ser: ser / cars_data1.shape[0] > 0.03].plot.pie(label="", title="Соотношение продаваемых машин по маркам", autopct='%.2f', figsize=(6, 6));

## Группировки

Распределение по возрастным категориям для топ 5 фирм по числу объявлений, чтобы не нагромождать вывод

In [ ]:
ser = cars_data1['Mark'].value_counts()
mark_list = ser.iloc[:5].index.tolist()
grouped_data = cars_data1[cars_data1['Mark'].isin(mark_list)].groupby(['Mark', 'Возраст']).size()
grouped_data

Посмотрим информацию по пробегу за год для всех автомобилей

In [ ]:
cars_data1['Пробег за год'].describe()

Посмотрим зависимость пробега машины от числа ее владельцев

In [ ]:
cars_data1.groupby(['Владельцы']).mean()['Пробег']

Как видно, чем больше владельцев было у машины, тем больше ее пробег

Посмотрим на зависимости состояния машины от числа ее владельцев

In [ ]:
cars_data1.groupby(['Владельцы', 'Состояние']).size()

Эти данные ничего нам не дали

Посмотрим на общее число просмотров объявлений в рамках фирмы. Вывели топ 5, чтобы не нагромождать вывод

In [ ]:
ser = cars_data1['Mark'].value_counts()
mark_list = ser.iloc[:5].index.tolist()
grouped_data = cars_data1[cars_data1['Mark'].isin(mark_list)].groupby(['Mark']).sum()['Total views'].sort_values(ascending=False)
grouped_data

Как видно, мерседес сильно лидирует
Посмотрим также в процентах

In [ ]:
grouped_data.to_frame()['Total views'] / grouped_data.sum() * 100

Как видно, более трети объявлений приходится на мерседес!